# Unsupervised neural-network solvers for multi-material Riemann problem


## Two-material Riemann problem for the 1D Euler equations

$$
 \frac{\partial U}{\partial t}+\frac{\partial F}{\partial x}=0,
 \quad U|_{t=t^n}=\left\{
   \begin{array}{ll}
     U_{L},& x<x_{I},\\
     U_{R},& x>x_{I}.
   \end{array}
   \right.
$$

$U=[\rho,\rho u,E]^{T}$, $F=[\rho u,\rho u^2+p,(E+p)u]^{T}$ are the vectors of conserved variables and fluxes. $\rho$, $u$ and $p$ are the density, velocity and pressure, respectively. $E$ is the total energy per unit volume and written as $E=\rho e+\rho u^2/2$, where $e$ is the internal energy per unit mass. $U_{L}$ and $U_{R}$ are two constant states separated by the material interface located at $x_{I}$. The subscripts I, L and R indicate the interface and the flow states on the left- and right-hand material, respectively.


## Stiffened gas EOS
$$
e=\frac{p+\Gamma B}{(\Gamma-1)\rho}
$$

If $\Gamma$ and $B$ are respectively set to $\gamma$ and 0, the perfect gas law is recovered. If the values of $\Gamma$ and $B$ are different on both sides of the interface, such EOS can be used to describe various multi-material problems involving gas-gas and gas-liquid interactions. The sound speed here can be expressed as $c=\sqrt{\Gamma(p+B)/\rho}$.

In [ ]:
import torch
import math
import copy
import numpy as np
from torch import nn
from torch.utils import data
import matplotlib.pyplot as plt

In [ ]:
# training dataset
int_data = np.loadtxt('data_interface-aw.dat',skiprows = 1)
np.random.shuffle(int_data)
int_data = torch.FloatTensor(int_data)

# validation dataset
int_data_test = np.loadtxt('data_interface-aw-validation.dat',skiprows = 1)
int_data_test = torch.FloatTensor(int_data_test)

## Neural network

Input layer: para_input = 5（$p_L, \rho_L, p_R, \rho_R, \Delta u$）

Output layer: para_output = 1（$p_I$）
 
Hidden layers: para_hid1, para_hid2, para_hid3, ...

Note that this is an unsupervised learning approach, and data labels are not used for training neural networks. Therefore, labels are not actually needed, but can be used to check the accuracy of the predicted results.

In [ ]:
para_input  = 5
para_output = 1
para_hid1   = 20
para_hid2   = 20
file_error  = f"PCNN-RS-{para_input}-{para_hid1}-{para_hid2}-{para_output}"

# Input training data
features_train, labels_train = int_data[:, 0:para_input], int_data[:, para_input : para_input + para_output]

# Input validation data
features_test, labels_test = int_data_test[:, 0:para_input], int_data_test[:, para_input : para_input + para_output]

In [ ]:
def load_array(data_arrays, batch_size, is_train=True): 
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train, drop_last=False)

batch_size = 256 
data_iter = load_array((features_train, labels_train), batch_size)
next(iter(data_iter))

## Neural network architecture


In [ ]:
net = nn.Sequential(nn.Linear(para_input, para_hid1), torch.nn.Tanh(), 
                    nn.Linear(para_hid1,  para_hid2), torch.nn.Tanh(), 
                    nn.Linear(para_hid2, para_output),torch.nn.Softplus())

def init_weights(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, std=0.01)

net.apply(init_weights);

loss = nn.MSELoss()

# Physical constraints in neural networks


## Description of symbols

| Physical quantity    | $p_L$      | $\rho_L$  | $p_R$  | $\rho_R$ |  $\Delta u$ | $p_I$    | 
| ----     |  ----      |  ----     |  ----   | ----     |  ----       |   ----  |  
| Definition     |  $x[:,0]$  | $x[:,1]$  | $x[:,2]$| $x[:,3]$ | $x[:,4]$ | $yyy[:,0]$ | 


## EOS parameters

"gammaL", "BL", "gammaR" and "BR" in this code are left and right parameters that need to be determined based on the specific problem, For example, 

(1) "gammaL=1.4", "BL=0.0" and "gammaR=1.667", "BR=0.0" are used for compressible air-helium flows. 

(2) "gammaL=1.4", "BL=0.0" and "gammaR=4.4", "BR=1.0" are used for compressible air-water flows. ("BR=1.0" is obtained based on the data transformation to improve the applicability of the surrogate model.)  

In [ ]:
trainer = torch.optim.Adam(net.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-06, weight_decay=0)
num_epochs = 40000

gammaL = 1.4
gammaR = 4.4
gammaL1= 1.0/gammaL
gammaR1= 1.0/gammaR
gammaL2= 2.0/(gammaL-1.0)
gammaR2= 2.0/(gammaR-1.0)
gammaLr= (gammaL-1.0)/(gammaL+1.0)
gammaRr= (gammaR-1.0)/(gammaR+1.0)
gammaLm1 = gammaL-1.0
gammaRm1 = gammaR-1.0
gammaLp1 = gammaL+1.0
gammaRp1 = gammaR+1.0

BL     = 0.0
BR     = 1.0

epsilon=1e-06  

val0 = torch.tensor([0.0])
val1 = torch.tensor([1.0])

plt.figure(1, figsize=(16, 6)) 
file_error_time = file_error+"-loss-error-time.dat"

# The four parameters of the output are, in order, epoch number, training MSE, minimum training MSE and validation error.
with open(file_error_time,'w') as f:                                                                                                                                                                                           
    f.write('Variables="Epoch","Loss","minLoss","Loss_val" \n')

lmin = 100000

for epoch in range(num_epochs):
    for x, _ in data_iter:
        
        # Output: interfacial pressure
        yyy=net(x)
        
        stepsL= torch.heaviside(yyy[:,0]-x[:,0], val1).detach()
        steprL= torch.heaviside(x[:,0]-yyy[:,0], val0).detach()
        stepsR= torch.heaviside(yyy[:,0]-x[:,2], val1).detach()
        steprR= torch.heaviside(x[:,2]-yyy[:,0], val0).detach()
      
        # specific volume
        nuL   = 1.0/x[:,1]
        nuR   = 1.0/x[:,3]
        nuIL  = stepsL[:] * nuL[:] * (gammaLm1*(yyy[:,0]+BL) + gammaLp1*(x[:,0]+BL)) / (gammaLp1*(yyy[:,0]+BL)+ gammaLm1*(x[:,0]+BL)) +  \
                steprL[:] * nuL[:] * torch.pow((x[:,0]+BL)/(yyy[:,0]+BL),gammaL1)
        nuIR  = stepsR[:] * nuR[:] * (gammaRm1*(yyy[:,0]+BR) + gammaRp1*(x[:,2]+BR)) / (gammaRp1*(yyy[:,0]+BR)+ gammaRm1*(x[:,2]+BR)) +  \
                steprR[:] * nuR[:] * torch.pow((x[:,2]+BR)/(yyy[:,0]+BR),gammaR1)

        # Physics-constrained functions
        fL = stepsL[:] * torch.pow((yyy[:,0]-x[:,0])*(nuL[:]-nuIL[:])+epsilon,0.5) +  \
             steprL[:] * gammaL2*(torch.pow(gammaL*(yyy[:,0]+BL)*nuIL[:], 0.5)-torch.pow(gammaL*(x[:,0]+BL)*nuL[:], 0.5))
        fR = stepsR[:] * torch.pow((yyy[:,0]-x[:,2])*(nuR[:]-nuIR[:])+epsilon,0.5) +  \
             steprR[:] * gammaR2*(torch.pow(gammaR*(yyy[:,0]+BR)*nuIR[:], 0.5)-torch.pow(gammaR*(x[:,2]+BR)*nuR[:], 0.5))

        yy = torch.zeros((len(x), para_output))

        yy[:,0] = x[:,4] + fL[:] + fR[:]

        y0 = torch.zeros((len(x), para_output))
        
        l = loss(yy, y0)

        trainer.zero_grad()
        l.backward()
        trainer.step()

    x = features_train
    
    # Output: interfacial pressure
    yyy=net(x)

    stepsL= torch.heaviside(yyy[:,0]-x[:,0], val1).detach()
    steprL= torch.heaviside(x[:,0]-yyy[:,0], val0).detach()
    stepsR= torch.heaviside(yyy[:,0]-x[:,2], val1).detach()
    steprR= torch.heaviside(x[:,2]-yyy[:,0], val0).detach()
 
    # specific volume
    nuL   = 1.0/x[:,1]
    nuR   = 1.0/x[:,3]
    nuIL  = stepsL[:] * nuL[:] * (gammaLm1*(yyy[:,0]+BL) + gammaLp1*(x[:,0]+BL)) / (gammaLp1*(yyy[:,0]+BL)+ gammaLm1*(x[:,0]+BL)) +  \
            steprL[:] * nuL[:] * torch.pow((x[:,0]+BL)/(yyy[:,0]+BL),gammaL1)
    nuIR  = stepsR[:] * nuR[:] * (gammaRm1*(yyy[:,0]+BR) + gammaRp1*(x[:,2]+BR)) / (gammaRp1*(yyy[:,0]+BR)+ gammaRm1*(x[:,2]+BR)) +  \
            steprR[:] * nuR[:] * torch.pow((x[:,2]+BR)/(yyy[:,0]+BR),gammaR1)

    # Physics-constrained functions
    fL = stepsL[:] * torch.pow((yyy[:,0]-x[:,0])*(nuL[:]-nuIL[:])+epsilon,0.5) +  \
         steprL[:] * gammaL2*(torch.pow(gammaL*(yyy[:,0]+BL)*nuIL[:], 0.5)-torch.pow(gammaL*(x[:,0]+BL)*nuL[:], 0.5))
    fR = stepsR[:] * torch.pow((yyy[:,0]-x[:,2])*(nuR[:]-nuIR[:])+epsilon,0.5) +  \
         steprR[:] * gammaR2*(torch.pow(gammaR*(yyy[:,0]+BR)*nuIR[:], 0.5)-torch.pow(gammaR*(x[:,2]+BR)*nuR[:], 0.5))
        
    yy = torch.zeros((len(x), para_output))

    yy[:,0] = x[:,4] + fL[:] + fR[:]
     
    y0      = torch.zeros((len(x), para_output))
    
    l       = loss(yy, y0)

    # validation
    x = features_test
    
    # Output: interfacial pressure
    yyy=net(x)

    stepsL= torch.heaviside(yyy[:,0]-x[:,0], val1).detach()
    steprL= torch.heaviside(x[:,0]-yyy[:,0], val0).detach()
    stepsR= torch.heaviside(yyy[:,0]-x[:,2], val1).detach()
    steprR= torch.heaviside(x[:,2]-yyy[:,0], val0).detach()
 
    # specific volume
    nuL   = 1.0/x[:,1]
    nuR   = 1.0/x[:,3]
    nuIL  = stepsL[:] * nuL[:] * (gammaLm1*(yyy[:,0]+BL) + gammaLp1*(x[:,0]+BL)) / (gammaLp1*(yyy[:,0]+BL)+ gammaLm1*(x[:,0]+BL)) +  \
            steprL[:] * nuL[:] * torch.pow((x[:,0]+BL)/(yyy[:,0]+BL),gammaL1)
    nuIR  = stepsR[:] * nuR[:] * (gammaRm1*(yyy[:,0]+BR) + gammaRp1*(x[:,2]+BR)) / (gammaRp1*(yyy[:,0]+BR)+ gammaRm1*(x[:,2]+BR)) +  \
            steprR[:] * nuR[:] * torch.pow((x[:,2]+BR)/(yyy[:,0]+BR),gammaR1)

    # Physics-constrained functions
    fL = stepsL[:] * torch.pow((yyy[:,0]-x[:,0])*(nuL[:]-nuIL[:])+epsilon,0.5) +  \
         steprL[:] * gammaL2*(torch.pow(gammaL*(yyy[:,0]+BL)*nuIL[:], 0.5)-torch.pow(gammaL*(x[:,0]+BL)*nuL[:], 0.5))
    fR = stepsR[:] * torch.pow((yyy[:,0]-x[:,2])*(nuR[:]-nuIR[:])+epsilon,0.5) +  \
         steprR[:] * gammaR2*(torch.pow(gammaR*(yyy[:,0]+BR)*nuIR[:], 0.5)-torch.pow(gammaR*(x[:,2]+BR)*nuR[:], 0.5))
        
    yy = torch.zeros((len(x), para_output))

    yy[:,0] = x[:,4] + fL[:] + fR[:]

    y0      = torch.zeros((len(x), para_output))
    
    l_test  = loss(yy, y0)

    if l < lmin :
        lmin    = l
        net_0   = copy.deepcopy(net)
        torch.save(net_0,f"{file_error}-minloss.pth")
        
    # The four parameters of the output are, in order, epoch number, training MSE, minimum training MSE and validation error.
    print(f'epoch {epoch + 1}, loss {l:.8f}, minloss {lmin:.8f}, valloss {l_test:.8f},')
    
    with open(file_error_time,'a') as f:                                                                                                                                                                                           
        f.write(f'{epoch + 1} {l:.8f} {lmin:.8f} {l_test:.8f}\n')
        
    plt.subplot(121)
    plt.scatter(epoch, l.detach().numpy(), c='red', label='train loss')
    plt.axis([0, num_epochs, 0, 100])
    plt.subplot(122)
    plt.scatter(epoch, l.detach().numpy(), c='red', label='train loss')
    plt.axis([0, num_epochs, 0, 10])
    
plt.draw()

torch.save(net,f"{file_error}-epoch-{num_epochs}.pth")
